In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import numpy as np
import cupy as cp
import copy
import pandas as pd
import glob

local


In [2]:
def find_legalized(N_trees, save_result=True):
    try: 
        import kaggle_support as kgs
        import pack_io
        import numpy as np
        kgs.set_float32(False)
        kgs.TREE_EXPANSION = kgs.TREE_EXPANSION**2
        kgs.initialize_tree_globals()
        xyt_list = kgs.dill_load(kgs.code_dir + '../res/sols.pickle')[N_trees-1]
        sol_list = []
        scores = []
        for xyt in xyt_list:
            sol = kgs.SolutionCollectionSquare()
            sol.xyt = xyt.astype(kgs.dtype_cp)        
            try:
                _,score = pack_io.solution_list_to_dataframe([sol], return_scores= True, print_score=False)
                scores.append(score[0])
            except:
                #print('failed to score solution')
                scores.append(np.nan)                
            sol_list.append(sol)
        #print(scores)
        min_score_before = np.nanmin(scores)
        #_,scores_test = pack_io.solution_list_to_dataframe(sol_list, return_scores= True, print_score=False)
        #print(scores_test)
        sol_list = [sol for sol, score in zip(sol_list, scores) if (np.isnan(score) or score <= min_score_before + 1e-3)]
        #_,scores_test = pack_io.solution_list_to_dataframe(sol_list, return_scores= True, print_score=False)
        #print(scores_test)
        print(f"{N_trees}: Considering {len(sol_list)} solutions")
        sol_list_legalized = []
        for sol in sol_list:
            #pack_io.solution_list_to_dataframe([sol], return_scores= True, print_score=True)
            sol_legalized = pack_io.legalize(sol, do_plot = False, validate=True, target = 1e-20, n_iter = 100, tolerance_rel_change=1e-10, line_search = False, verbose=False, scaling=1.)
            sol_list_legalized.append(sol_legalized)
            #pack_io.solution_list_to_dataframe([sol_legalized], return_scores= True, print_score=True)
        _,scores = pack_io.solution_list_to_dataframe(sol_list_legalized, return_scores= True, print_score=False)
        print(f"{N_trees}: Score before legalize: {min_score_before:.8f}")
        print(f"{N_trees}: Score after legalize: {np.nanmin(scores):.8f}")
        if save_result:
            kgs.dill_save(kgs.code_dir + f'../res/sol_legalized_{N_trees}.pickle', sol_list_legalized[np.argmin(scores)])
        return sol_list_legalized[np.argmin(scores)]
    except Exception as e:
        print(f"Error in legalizing for N_trees={N_trees}: {e}")
        raise e

In [3]:
find_legalized(41)

41: Considering 3 solutions
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENABLE_SEPARATION -cubin /mnt/d//packing/temp/pack_cuda_saved_32368.cu -o /mnt/d//packing/temp/pack_cuda_crystal_32368.cubin
ptxas info    : 144 bytes gmem, 2324 bytes cmem[3], 8 bytes cmem[4]
ptxas info    : Compiling entry function 'multi_boundary_distance_list_total' for 'sm_89'
ptxas info    : Function properties for multi_boundary_distance_list_total
    88 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 38 registers, used 1 barriers, 88 bytes cumulative stack size, 404 bytes cmem[0], 120 bytes cmem[2]
ptxas info    : Compile time = 15.052 ms
ptxas

SolutionCollectionSquare(comment='', xyt=array([[[-6.75766878e-02,  9.90162589e-01,  1.89284509e+00],
        [-9.10881927e-01, -4.65636067e-01,  2.39170334e+01],
        [ 1.60997730e+00, -1.93360808e-01,  1.20627635e+01],
        [-2.17120284e-01,  1.62294715e+00, -5.19047963e+00],
        [ 7.58605503e-01, -1.63063435e+00, -2.03419538e+00],
        [-1.61837415e+00, -8.07071238e-01,  1.52283523e+01],
        [-1.48087279e+00, -1.51315366e-01,  5.87139144e+00],
        [-3.76323306e-02,  6.17760560e-02,  5.04242978e+00],
        [-1.65730362e+00,  3.18159483e-01,  1.52961694e+01],
        [-1.65751457e+00,  1.25935537e+00,  6.69559575e+00],
        [ 1.17759536e+00, -1.25712497e+00,  4.97685158e+00],
        [-1.26799496e+00,  8.56663105e-01,  1.21931647e+01],
        [ 1.46739574e+00,  4.94192821e-01,  8.18291514e+00],
        [ 5.89144237e-01,  1.04811877e+00,  5.02757678e+00],
        [ 5.63287486e-01, -6.18898430e-03,  1.86699243e+00],
        [ 8.96583425e-01,  5.49149849e-01,  

In [ ]:
# Run legalize for N=1 to 200
import multiprocess as mp

if __name__ == '__main__':
    # Create a pool of 10 workers
    with mp.Pool(processes=10) as pool:
        # Run find_legalized for N=1 to 200 in parallel
        pool.map(find_legalized, range(1, 201))
    
    print("All legalizations completed!")

local
local
local
local
local
local
local
local
local
local
1: Considering 7 solutions
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENABLE_SEPARATION -cubin /mnt/d//packing/temp/pack_cuda_saved_34722.cu -o /mnt/d//packing/temp/pack_cuda_crystal_34722.cubin
6: Considering 7 solutions
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENA